
# Introdução

## Sistemas de Recomendação 

Hoje em dia, com a facilidade de buscar coisas na internet, assistir filmes online, entre outras coisas, os sistemas de recomendação começaram a ganhar muita importância. 

Irei construir um modelo que, dado uma matriz de usuários e notas de filmes, o sistema irá recomendar um filme para uma determinada pessoa dessa lista. 

Vou utilizar técnicas de filtros colaborativos baseados em modelos e em memórias. Para isso, precisarei calcular as similaridades, portanto irei precisar das distâncias euclidianas.

O Dataset que irei utilizar contém 15 usuários e 8 filmes, com notas variando de 1 a 10. 

Um ponto importante de se notar é que nem todos os usuários viram todos os filmes.
 





In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [3]:
#DATA FRAME
def getDF():
    dic__ = {"User_1":[np.nan, np.nan, np.nan, 1, 7, 2, 3, 8],
         "User_2":[9,10,2,2,6,5,3,8],
         "User_3":[4, 7, 9, 6,6,10,10,2],
         "User_4":[np.nan, 7, 9, 5, 5, 10, 9, 1],
         "User_5":[7.0,6.0,3.0,8.0,3,4.0,3.0, 2],
         "User_6":[np.nan, np.nan, 9, 9,6,8,9,np.nan],
         "User_7":[3,5,4,4,3,3,9,np.nan],
         "User_8":[10,10,10,10,2,2,2,2],
         "User_9":[9,9,np.nan,8,3,3,1,np.nan],
         "User_10":[9,8,10,9,3,4,2,1],
         "User_11":[4,4,3,3,9,9,8,10],
         "User_12":[2,2,4,1,8,10,10,9],
         "User_13":[1,4,1,3,7,10,7,8],
         "User_14":[3,3,2,1,1,10, np.nan,10],
         "User_15":[9,9,8,10,4,2,np.nan,1]
        }
    df = pd.DataFrame(dic__).T
    df.columns = ['Filme_'+str(int(i+1)) for i in range(8)]
    return df

<br>

__Matriz de interação usuário/item:__

In [4]:
getDF()

,Filme_1,Filme_2,Filme_3,Filme_4,Filme_5,Filme_6,Filme_7,Filme_8
User_1,NaN,NaN,NaN,1.0,7.0,2.0,3.0,8.0
User_2,9.0,10.0,2.0,2.0,6.0,5.0,3.0,8.0
User_3,4.0,7.0,9.0,6.0,6.0,10.0,10.0,2.0
User_4,NaN,7.0,9.0,5.0,5.0,10.0,9.0,1.0
User_5,7.0,6.0,3.0,8.0,3.0,4.0,3.0,2.0
User_6,NaN,NaN,9.0,9.0,6.0,8.0,9.0,NaN
User_7,3.0,5.0,4.0,4.0,3.0,3.0,9.0,NaN
User_8,10.0,10.0,10.0,10.0,2.0,2.0,2.0,2.0
User_9,9.0,9.0,NaN,8.0,3.0,3.0,1.0,NaN
User_10,9.0,8.0,10.0,9.0,3.0,4.0,2.0,1.0


In [5]:
from sklearn.metrics.pairwise import nan_euclidean_distances

def euclid_dist(x, y):
    v = np.vstack([x, y])
    if np.isnan(nan_euclidean_distances(v)[0,1]):
        return 1e10
    else:
        return nan_euclidean_distances(v)[0,1]

Criei uma classe de similaridade para facilitar

In [11]:
class Similaridade():
    
    def __init__(self, dataframe):
        self.df = dataframe
        self.lista_usuarios = dataframe.index.tolist()
        self.lista_filmes = dataframe.columns.tolist()
        
    def similaridadeUsuarios(self, user, sorted_list = True):
        nota_user = self.df.loc[user, :].values 
        L = []
        for u in self.lista_usuarios:
            if u == user:
                continue
            else:
                nota_u = self.df.loc[u, :].values
                dist = euclid_dist(nota_user, nota_u)
                similaridade = 1 / (1 + dist)
                L.append([u,similaridade])
        if sorted_list:
            L.sort(key = lambda s: s[1], reverse = True)
            return L
        else:
            return L

    def similaridadeFilmes(self, filme, sorted_list = True): 
        nota_filme = self.df.T.loc[filme, :].values
        L = []
        for f in self.lista_filmes: 
            if f == filme:
                continue
            else:
                nota_f = self.df.T.loc[f, :].values
                dist = euclid_dist(nota_filme, nota_f)
                similaridade = 1 / (1 + dist)
                L.append([f,similaridade])
        if sorted_list:
            L.sort(key = lambda s: s[1], reverse = True)
            return L
        else:
            return L
        
    def similaridade_entre_dois_usuarios(self, user1, user2):
            return [x[1] for x in self.similaridadeUsuarios(user1) if x[0] == user2 ][0]

    def similaridade_entre_dois_filmes(self, filme1, filme2):
        return [x[1] for x in self.similaridadeFilmes(filme1) if x[0] == filme2 ][0]

In [12]:
df = getDF()

In [13]:
df

,Filme_1,Filme_2,Filme_3,Filme_4,Filme_5,Filme_6,Filme_7,Filme_8
User_1,NaN,NaN,NaN,1.0,7.0,2.0,3.0,8.0
User_2,9.0,10.0,2.0,2.0,6.0,5.0,3.0,8.0
User_3,4.0,7.0,9.0,6.0,6.0,10.0,10.0,2.0
User_4,NaN,7.0,9.0,5.0,5.0,10.0,9.0,1.0
User_5,7.0,6.0,3.0,8.0,3.0,4.0,3.0,2.0
User_6,NaN,NaN,9.0,9.0,6.0,8.0,9.0,NaN
User_7,3.0,5.0,4.0,4.0,3.0,3.0,9.0,NaN
User_8,10.0,10.0,10.0,10.0,2.0,2.0,2.0,2.0
User_9,9.0,9.0,NaN,8.0,3.0,3.0,1.0,NaN
User_10,9.0,8.0,10.0,9.0,3.0,4.0,2.0,1.0


In [14]:
sim = Similaridade(dataframe = df)

<br>

__Exemplificando algumas funcionalidades dessa classe:__

In [15]:
sim.df

,Filme_1,Filme_2,Filme_3,Filme_4,Filme_5,Filme_6,Filme_7,Filme_8
User_1,NaN,NaN,NaN,1.0,7.0,2.0,3.0,8.0
User_2,9.0,10.0,2.0,2.0,6.0,5.0,3.0,8.0
User_3,4.0,7.0,9.0,6.0,6.0,10.0,10.0,2.0
User_4,NaN,7.0,9.0,5.0,5.0,10.0,9.0,1.0
User_5,7.0,6.0,3.0,8.0,3.0,4.0,3.0,2.0
User_6,NaN,NaN,9.0,9.0,6.0,8.0,9.0,NaN
User_7,3.0,5.0,4.0,4.0,3.0,3.0,9.0,NaN
User_8,10.0,10.0,10.0,10.0,2.0,2.0,2.0,2.0
User_9,9.0,9.0,NaN,8.0,3.0,3.0,1.0,NaN
User_10,9.0,8.0,10.0,9.0,3.0,4.0,2.0,1.0


In [16]:
sim.lista_usuarios

['User_1',
 'User_2',
 'User_3',
 'User_4',
 'User_5',
 'User_6',
 'User_7',
 'User_8',
 'User_9',
 'User_10',
 'User_11',
 'User_12',
 'User_13',
 'User_14',
 'User_15']

In [17]:
sim.lista_filmes

['Filme_1',
 'Filme_2',
 'Filme_3',
 'Filme_4',
 'Filme_5',
 'Filme_6',
 'Filme_7',
 'Filme_8']

<br>

__Calculando a similaridade do User_1 com os outros usuários:__

In [18]:
sim.similaridadeUsuarios("User_1")

[['User_2', 0.1924840598000365],
 ['User_7', 0.08240267256634183],
 ['User_13', 0.07940855619903711],
 ['User_11', 0.07855272795675068],
 ['User_9', 0.0779292055122247],
 ['User_5', 0.07162563710668096],
 ['User_12', 0.06865934407787179],
 ['User_14', 0.06484157054036695],
 ['User_10', 0.06392879181901828],
 ['User_8', 0.06201113398134867],
 ['User_4', 0.057326814524408215],
 ['User_6', 0.05697049581409102],
 ['User_15', 0.056582425994704214],
 ['User_3', 0.05639139975154663]]

Vemos que o usuário mais similar ao usuário 1 foi o usuário 2. Além disso, o usuário menos similar foi o usuário 3.

Abaixo, as comparações das notas que esses usuários tiveram:

In [19]:
df.loc[['User_1', 'User_2']]

,Filme_1,Filme_2,Filme_3,Filme_4,Filme_5,Filme_6,Filme_7,Filme_8
User_1,NaN,NaN,NaN,1.0,7.0,2.0,3.0,8.0
User_2,9.0,10.0,2.0,2.0,6.0,5.0,3.0,8.0


In [20]:
df.loc[['User_1', 'User_3']]

,Filme_1,Filme_2,Filme_3,Filme_4,Filme_5,Filme_6,Filme_7,Filme_8
User_1,NaN,NaN,NaN,1.0,7.0,2.0,3.0,8.0
User_3,4.0,7.0,9.0,6.0,6.0,10.0,10.0,2.0


<br>

Fazendo agora o mesmo, mas analisando as similaridades dos filmes (itens):

In [21]:
sim.similaridadeFilmes("Filme_8")

[['Filme_5', 0.07147768141142521],
 ['Filme_6', 0.058133276275409755],
 ['Filme_7', 0.05648666578593548],
 ['Filme_2', 0.04067130131368077],
 ['Filme_1', 0.03899013204902304],
 ['Filme_3', 0.035830816042996094],
 ['Filme_4', 0.035779422208098904]]

In [22]:
df.T

,User_1,User_2,User_3,User_4,User_5,User_6,User_7,User_8,User_9,User_10,User_11,User_12,User_13,User_14,User_15
Filme_1,NaN,9.0,4.0,NaN,7.0,NaN,3.0,10.0,9.0,9.0,4.0,2.0,1.0,3.0,9.0
Filme_2,NaN,10.0,7.0,7.0,6.0,NaN,5.0,10.0,9.0,8.0,4.0,2.0,4.0,3.0,9.0
Filme_3,NaN,2.0,9.0,9.0,3.0,9.0,4.0,10.0,NaN,10.0,3.0,4.0,1.0,2.0,8.0
Filme_4,1.0,2.0,6.0,5.0,8.0,9.0,4.0,10.0,8.0,9.0,3.0,1.0,3.0,1.0,10.0
Filme_5,7.0,6.0,6.0,5.0,3.0,6.0,3.0,2.0,3.0,3.0,9.0,8.0,7.0,1.0,4.0
Filme_6,2.0,5.0,10.0,10.0,4.0,8.0,3.0,2.0,3.0,4.0,9.0,10.0,10.0,10.0,2.0
Filme_7,3.0,3.0,10.0,9.0,3.0,9.0,9.0,2.0,1.0,2.0,8.0,10.0,7.0,NaN,NaN
Filme_8,8.0,8.0,2.0,1.0,2.0,NaN,NaN,2.0,NaN,1.0,10.0,9.0,8.0,10.0,1.0


In [23]:
df.T.loc[['Filme_8', 'Filme_5']]

,User_1,User_2,User_3,User_4,User_5,User_6,User_7,User_8,User_9,User_10,User_11,User_12,User_13,User_14,User_15
Filme_8,8.0,8.0,2.0,1.0,2.0,NaN,NaN,2.0,NaN,1.0,10.0,9.0,8.0,10.0,1.0
Filme_5,7.0,6.0,6.0,5.0,3.0,6.0,3.0,2.0,3.0,3.0,9.0,8.0,7.0,1.0,4.0


In [24]:
df.T.loc[['Filme_8', 'Filme_4']]

,User_1,User_2,User_3,User_4,User_5,User_6,User_7,User_8,User_9,User_10,User_11,User_12,User_13,User_14,User_15
Filme_8,8.0,8.0,2.0,1.0,2.0,NaN,NaN,2.0,NaN,1.0,10.0,9.0,8.0,10.0,1.0
Filme_4,1.0,2.0,6.0,5.0,8.0,9.0,4.0,10.0,8.0,9.0,3.0,1.0,3.0,1.0,10.0






## Filtros colaborativos baseados em usuários



In [25]:
class InfoRec():
    
    def __init__(self, dataframe):
        self.df = dataframe
        
    def getFilmes(self, user):
        return self.df.loc[user]

    def getUsers(self, filme):
        return self.df.T.loc[filme]

    def getPossibleMovies(self, user):
        return self.getFilmes(user)[np.isnan(self.getFilmes(user))].index.tolist()

    def getPossibleUsers(self, filme):
        return self.getUsers(filme)[np.isnan(self.getUsers(filme))].index.tolist()

In [26]:
df

,Filme_1,Filme_2,Filme_3,Filme_4,Filme_5,Filme_6,Filme_7,Filme_8
User_1,NaN,NaN,NaN,1.0,7.0,2.0,3.0,8.0
User_2,9.0,10.0,2.0,2.0,6.0,5.0,3.0,8.0
User_3,4.0,7.0,9.0,6.0,6.0,10.0,10.0,2.0
User_4,NaN,7.0,9.0,5.0,5.0,10.0,9.0,1.0
User_5,7.0,6.0,3.0,8.0,3.0,4.0,3.0,2.0
User_6,NaN,NaN,9.0,9.0,6.0,8.0,9.0,NaN
User_7,3.0,5.0,4.0,4.0,3.0,3.0,9.0,NaN
User_8,10.0,10.0,10.0,10.0,2.0,2.0,2.0,2.0
User_9,9.0,9.0,NaN,8.0,3.0,3.0,1.0,NaN
User_10,9.0,8.0,10.0,9.0,3.0,4.0,2.0,1.0


In [27]:
info = InfoRec(dataframe = df)

In [28]:
info.getFilmes("User_1")

Filme_1    NaN
Filme_2    NaN
Filme_3    NaN
Filme_4    1.0
Filme_5    7.0
Filme_6    2.0
Filme_7    3.0
Filme_8    8.0
Name: User_1, dtype: float64

In [29]:
info.getUsers("Filme_1")

User_1      NaN
User_2      9.0
User_3      4.0
User_4      NaN
User_5      7.0
User_6      NaN
User_7      3.0
User_8     10.0
User_9      9.0
User_10     9.0
User_11     4.0
User_12     2.0
User_13     1.0
User_14     3.0
User_15     9.0
Name: Filme_1, dtype: float64

In [30]:
info.getPossibleMovies("User_1")

['Filme_1', 'Filme_2', 'Filme_3']

In [31]:
info.getPossibleUsers("Filme_1")

['User_1', 'User_4', 'User_6']

<br>

__Aplicando esta fórmula para um usuário específico:__

<br>

In [36]:
#fixando um usuário específico

user = "User_1"

In [38]:
#calculando a similaridade de todos os outros usuários com este usuário específico

df1 = pd.DataFrame(sim.similaridadeUsuarios(user, sorted_list = False), 
                   columns = ['usuario', 'similaridade'])
df1

,usuario,similaridade
0,User_2,0.192484
1,User_3,0.056391
2,User_4,0.057327
3,User_5,0.071626
4,User_6,0.056970
5,User_7,0.082403
6,User_8,0.062011
7,User_9,0.077929
8,User_10,0.063929
9,User_11,0.078553


In [39]:
#coletando, para cada um dos outros usuários, as notas que eles deram para os filmes

df2 = sim.df.loc[df1.usuario, ].reset_index().rename(columns = {"index":"usuario"})
df2

,usuario,Filme_1,Filme_2,Filme_3,Filme_4,Filme_5,Filme_6,Filme_7,Filme_8
0,User_2,9.0,10.0,2.0,2.0,6.0,5.0,3.0,8.0
1,User_3,4.0,7.0,9.0,6.0,6.0,10.0,10.0,2.0
2,User_4,NaN,7.0,9.0,5.0,5.0,10.0,9.0,1.0
3,User_5,7.0,6.0,3.0,8.0,3.0,4.0,3.0,2.0
4,User_6,NaN,NaN,9.0,9.0,6.0,8.0,9.0,NaN
5,User_7,3.0,5.0,4.0,4.0,3.0,3.0,9.0,NaN
6,User_8,10.0,10.0,10.0,10.0,2.0,2.0,2.0,2.0
7,User_9,9.0,9.0,NaN,8.0,3.0,3.0,1.0,NaN
8,User_10,9.0,8.0,10.0,9.0,3.0,4.0,2.0,1.0
9,User_11,4.0,4.0,3.0,3.0,9.0,9.0,8.0,10.0


In [40]:
#agrupando as duas tabelas acima numa única

df3 = df1.merge(df2, on = 'usuario', how = 'inner')
df3

,usuario,similaridade,Filme_1,Filme_2,Filme_3,Filme_4,Filme_5,Filme_6,Filme_7,Filme_8
0,User_2,0.192484,9.0,10.0,2.0,2.0,6.0,5.0,3.0,8.0
1,User_3,0.056391,4.0,7.0,9.0,6.0,6.0,10.0,10.0,2.0
2,User_4,0.057327,NaN,7.0,9.0,5.0,5.0,10.0,9.0,1.0
3,User_5,0.071626,7.0,6.0,3.0,8.0,3.0,4.0,3.0,2.0
4,User_6,0.056970,NaN,NaN,9.0,9.0,6.0,8.0,9.0,NaN
5,User_7,0.082403,3.0,5.0,4.0,4.0,3.0,3.0,9.0,NaN
6,User_8,0.062011,10.0,10.0,10.0,10.0,2.0,2.0,2.0,2.0
7,User_9,0.077929,9.0,9.0,NaN,8.0,3.0,3.0,1.0,NaN
8,User_10,0.063929,9.0,8.0,10.0,9.0,3.0,4.0,2.0,1.0
9,User_11,0.078553,4.0,4.0,3.0,3.0,9.0,9.0,8.0,10.0


In [41]:
#extraindo quais são os filmes possíveis para indicar para este usuário específico

info.getPossibleMovies(user)

['Filme_1', 'Filme_2', 'Filme_3']

In [42]:
#calculando as predições para cada um dos filmes disponíveis


#dicionario para armazenar as predições de cada filme
dic_recs = {}

#loop nos filmes disponíveis
for f in info.getPossibleMovies(user): 
    
    #acessando a coluna de similaridades
    sims = df3.similaridade
    
    #acessando a coluna de notas do filme dentro do loop
    nota_f = df3[f]
    
    #fazendo as contas, de acordo com a fórmula
    total_sim_vs_notafilme = (sims * nota_f.fillna(0)).sum()
    total_sims = sims[~sims.index.isin(nota_f.loc[np.isnan(nota_f)].index.tolist())].sum()
    
    #salvando a predição final de cada filme no dicionário
    dic_recs[f] = total_sim_vs_notafilme / total_sims
    
dic_recs

{'Filme_1': 6.114307160765262,
 'Filme_2': 6.786057983472178,
 'Filme_3': 4.940648740807049}

<br>

__Colocando isso numa classe python:__
    
<br>

In [43]:
class UserBasedColabFiltering():

    def __init__(self, dataframe):
        self.df = dataframe
        self.sim = Similaridade(dataframe = self.df)
        self.info = InfoRec(dataframe = self.df)
        
    def makeRec(self, user):
        df1 = pd.DataFrame(self.sim.similaridadeUsuarios(user, sorted_list = False), 
              columns = ['usuario', 'similaridade'])
        df2 = self.df.loc[df1.usuario, ].reset_index().rename(columns = {"index":"usuario"})
        df3 = df1.merge(df2, on = 'usuario', how = 'inner')
        dic_recs = {}
        for f in self.info.getPossibleMovies(user): 
            sims = df3.similaridade
            nota_f = df3[f]
            total_sim_vs_notafilme = (sims * nota_f.fillna(0)).sum()
            total_sims = sims[~sims.index.isin(nota_f.loc[np.isnan(nota_f)].index.tolist())].sum()
            dic_recs[f] = total_sim_vs_notafilme / total_sims
        return dic_recs

In [44]:
usb = UserBasedColabFiltering(dataframe = df)

In [45]:
usb.makeRec("User_1")

{'Filme_1': 6.114307160765262,
 'Filme_2': 6.786057983472178,
 'Filme_3': 4.940648740807049}

In [46]:
usb.makeRec("User_8")

{}

In [47]:
usb.makeRec("User_9")

{'Filme_3': 6.869332239924477, 'Filme_8': 3.4132070814421516}


<br>

Interpretando das recomendações para o usuário 9.

Vemos que esse usuário é bem similar aos usuários 10 e 15 e pouco similar aos usuários 13 e 12:

In [48]:
sim.similaridadeUsuarios("User_9")

[['User_10', 0.30216947925196225],
 ['User_15', 0.24399845080999227],
 ['User_8', 0.2240092377397959],
 ['User_5', 0.16952084719853724],
 ['User_2', 0.10542649822871226],
 ['User_1', 0.0779292055122247],
 ['User_7', 0.07009428092240914],
 ['User_6', 0.0663515090328441],
 ['User_4', 0.06484157054036695],
 ['User_3', 0.061943455133433856],
 ['User_11', 0.05825534265291749],
 ['User_14', 0.056544062334864344],
 ['User_13', 0.05576861189825108],
 ['User_12', 0.047468603961136634]]

<br>

Ao analisar a nota do filme 3 para esses usuários, vemos que os usuários similares ao User_8 deram notas altas para este filme. Por outro lado, usuários pouco similares ao User_8 deram notas baixas para o filme.

Por consequência, o algoritmo acaba gerando uma nota relativamente alta (perto de 7.0) para o User_8 neste filme!

<br>

In [49]:
df.loc[['User_10', 'User_15', 'User_13', 'User_12'], ]

,Filme_1,Filme_2,Filme_3,Filme_4,Filme_5,Filme_6,Filme_7,Filme_8
User_10,9.0,8.0,10.0,9.0,3.0,4.0,2.0,1.0
User_15,9.0,9.0,8.0,10.0,4.0,2.0,NaN,1.0
User_13,1.0,4.0,1.0,3.0,7.0,10.0,7.0,8.0
User_12,2.0,2.0,4.0,1.0,8.0,10.0,10.0,9.0






## Filtros colaborativos baseados em ITENS

In [50]:
#fixando um item (filme) específico

f = 'Filme_1'

In [51]:
#calculando a similaridade deste filme específico com todos os outros filmes

df1 = pd.DataFrame(sim.similaridadeFilmes(f, sorted_list = False), 
                   columns = ['filme', 'similaridade'])

df1

,filme,similaridade
0,Filme_2,0.151741
1,Filme_3,0.079246
2,Filme_4,0.098508
3,Filme_5,0.049820
4,Filme_6,0.040286
5,Filme_7,0.038447
6,Filme_8,0.038990


In [52]:
#coletando, para cada um dos outros filmes, as notas que os usuários deram

df2 = sim.df.T.loc[df1.filme, ].reset_index().rename(columns = {"index":"filme"})
df2

,filme,User_1,User_2,User_3,User_4,User_5,User_6,User_7,User_8,User_9,User_10,User_11,User_12,User_13,User_14,User_15
0,Filme_2,NaN,10.0,7.0,7.0,6.0,NaN,5.0,10.0,9.0,8.0,4.0,2.0,4.0,3.0,9.0
1,Filme_3,NaN,2.0,9.0,9.0,3.0,9.0,4.0,10.0,NaN,10.0,3.0,4.0,1.0,2.0,8.0
2,Filme_4,1.0,2.0,6.0,5.0,8.0,9.0,4.0,10.0,8.0,9.0,3.0,1.0,3.0,1.0,10.0
3,Filme_5,7.0,6.0,6.0,5.0,3.0,6.0,3.0,2.0,3.0,3.0,9.0,8.0,7.0,1.0,4.0
4,Filme_6,2.0,5.0,10.0,10.0,4.0,8.0,3.0,2.0,3.0,4.0,9.0,10.0,10.0,10.0,2.0
5,Filme_7,3.0,3.0,10.0,9.0,3.0,9.0,9.0,2.0,1.0,2.0,8.0,10.0,7.0,NaN,NaN
6,Filme_8,8.0,8.0,2.0,1.0,2.0,NaN,NaN,2.0,NaN,1.0,10.0,9.0,8.0,10.0,1.0


In [53]:
#agrupando as duas tabelas acima numa única

df3 = df1.merge(df2, on = 'filme', how = 'inner')
df3

,filme,similaridade,User_1,User_2,User_3,User_4,User_5,User_6,User_7,User_8,User_9,User_10,User_11,User_12,User_13,User_14,User_15
0,Filme_2,0.151741,NaN,10.0,7.0,7.0,6.0,NaN,5.0,10.0,9.0,8.0,4.0,2.0,4.0,3.0,9.0
1,Filme_3,0.079246,NaN,2.0,9.0,9.0,3.0,9.0,4.0,10.0,NaN,10.0,3.0,4.0,1.0,2.0,8.0
2,Filme_4,0.098508,1.0,2.0,6.0,5.0,8.0,9.0,4.0,10.0,8.0,9.0,3.0,1.0,3.0,1.0,10.0
3,Filme_5,0.049820,7.0,6.0,6.0,5.0,3.0,6.0,3.0,2.0,3.0,3.0,9.0,8.0,7.0,1.0,4.0
4,Filme_6,0.040286,2.0,5.0,10.0,10.0,4.0,8.0,3.0,2.0,3.0,4.0,9.0,10.0,10.0,10.0,2.0
5,Filme_7,0.038447,3.0,3.0,10.0,9.0,3.0,9.0,9.0,2.0,1.0,2.0,8.0,10.0,7.0,NaN,NaN
6,Filme_8,0.038990,8.0,8.0,2.0,1.0,2.0,NaN,NaN,2.0,NaN,1.0,10.0,9.0,8.0,10.0,1.0


In [54]:
#extraindo quais são os usuários possíveis para indicar para este filme específico

info.getPossibleUsers(f)

['User_1', 'User_4', 'User_6']

In [55]:
#calculando as predições para cada um dos usuários disponíveis


#dicionario para armazenar as predições de cada usuário
dic_recs = {}

#loop nos usuários disponíveis
for u in info.getPossibleUsers(f):
    
    #acessando a coluna de similaridades
    sims = df3.similaridade
    
    #acessando a coluna de notas do usuário dentro do loop
    nota_u = df3[u]
    
    #fazendo as contas, de acordo com a fórmula
    total_sim_vs_notafilme = (sims * nota_u.fillna(0)).sum()
    total_sims = sims[~sims.index.isin(nota_u.loc[np.isnan(nota_u)].index.tolist())].sum()
    
    #salvando a predição final de cada usuário no dicionário
    dic_recs[u] = total_sim_vs_notafilme / total_sims
    
dic_recs

{'User_1': 3.589846973408298,
 'User_4': 6.649218910791067,
 'User_6': 8.38053388736287}

<br>

__Colocando isso numa classe python:__
    
<br>

In [56]:
class ItemBasedColabFiltering():

    def __init__(self, dataframe):
        self.df = dataframe
        self.sim = Similaridade(dataframe = self.df)
        self.info = InfoRec(dataframe = self.df)
        
    def makeRec(self, filme):
        df1 = pd.DataFrame(self.sim.similaridadeFilmes(filme, sorted_list = False), 
                           columns = ['filme', 'similaridade'])
        df2 = self.df.T.loc[df1.filme, ].reset_index().rename(columns = {"index":"filme"})
        df3 = df1.merge(df2, on = 'filme', how = 'inner')
        dic_recs = {}
        for u in self.info.getPossibleUsers(filme):
            sims = df3.similaridade
            nota_u = df3[u]
            total_sim_vs_notafilme = (sims * nota_u.fillna(0)).sum()
            total_sims = sims[~sims.index.isin(nota_u.loc[np.isnan(nota_u)].index.tolist())].sum()
            dic_recs[u] = total_sim_vs_notafilme / total_sims
        return dic_recs

In [57]:
user_based = UserBasedColabFiltering(dataframe = df)
item_based = ItemBasedColabFiltering(dataframe = df)

In [58]:
df

,Filme_1,Filme_2,Filme_3,Filme_4,Filme_5,Filme_6,Filme_7,Filme_8
User_1,NaN,NaN,NaN,1.0,7.0,2.0,3.0,8.0
User_2,9.0,10.0,2.0,2.0,6.0,5.0,3.0,8.0
User_3,4.0,7.0,9.0,6.0,6.0,10.0,10.0,2.0
User_4,NaN,7.0,9.0,5.0,5.0,10.0,9.0,1.0
User_5,7.0,6.0,3.0,8.0,3.0,4.0,3.0,2.0
User_6,NaN,NaN,9.0,9.0,6.0,8.0,9.0,NaN
User_7,3.0,5.0,4.0,4.0,3.0,3.0,9.0,NaN
User_8,10.0,10.0,10.0,10.0,2.0,2.0,2.0,2.0
User_9,9.0,9.0,NaN,8.0,3.0,3.0,1.0,NaN
User_10,9.0,8.0,10.0,9.0,3.0,4.0,2.0,1.0


In [59]:
user_based.makeRec("User_1")

{'Filme_1': 6.114307160765262,
 'Filme_2': 6.786057983472178,
 'Filme_3': 4.940648740807049}

In [60]:
sim.similaridadeUsuarios("User_1")

[['User_2', 0.1924840598000365],
 ['User_7', 0.08240267256634183],
 ['User_13', 0.07940855619903711],
 ['User_11', 0.07855272795675068],
 ['User_9', 0.0779292055122247],
 ['User_5', 0.07162563710668096],
 ['User_12', 0.06865934407787179],
 ['User_14', 0.06484157054036695],
 ['User_10', 0.06392879181901828],
 ['User_8', 0.06201113398134867],
 ['User_4', 0.057326814524408215],
 ['User_6', 0.05697049581409102],
 ['User_15', 0.056582425994704214],
 ['User_3', 0.05639139975154663]]

In [61]:
df.loc[['User_1', 'User_2', 'User_7', 'User_13']]

,Filme_1,Filme_2,Filme_3,Filme_4,Filme_5,Filme_6,Filme_7,Filme_8
User_1,NaN,NaN,NaN,1.0,7.0,2.0,3.0,8.0
User_2,9.0,10.0,2.0,2.0,6.0,5.0,3.0,8.0
User_7,3.0,5.0,4.0,4.0,3.0,3.0,9.0,NaN
User_13,1.0,4.0,1.0,3.0,7.0,10.0,7.0,8.0




Como vemos, o usuário 1 é similar aos usuários 2, 7 e 13. Inclusive,  a similaridade do usuário 1 com o 2 é consideravelmente maior do que com os usuários 7 e 13. Como a nota que o usuários 2 deu para o filme 2 foi muito alta (10), isso acaba influenciando bastante na recomendação final!

<br>


In [62]:
item_based.makeRec("Filme_1")

{'User_1': 3.589846973408298,
 'User_4': 6.649218910791067,
 'User_6': 8.38053388736287}

In [63]:
sim.similaridadeFilmes("Filme_1")

[['Filme_2', 0.15174115516527187],
 ['Filme_4', 0.0985075843971186],
 ['Filme_3', 0.07924589581061381],
 ['Filme_5', 0.04982007201312378],
 ['Filme_6', 0.04028642276989089],
 ['Filme_8', 0.03899013204902304],
 ['Filme_7', 0.03844675420464307]]

Usando a filtragem baseada em itens, vemos que o Filme 1 é muito similar aos filmes 2, 3 e 4.

Analisando as notas que o usuário 6 deu para esses filmes:

In [64]:
df.loc[['User_6']]

,Filme_1,Filme_2,Filme_3,Filme_4,Filme_5,Filme_6,Filme_7,Filme_8
User_6,NaN,NaN,9.0,9.0,6.0,8.0,9.0,NaN


Vemos que apesar desse usuário não ter dado nota para o filme 2, ele deu ótimas notas para os filmes 3 e 4. Dessa forma, é esperado que a nota do filme 1 também seja alta
